In [1]:
import pandas as pd
import numpy as numpy
from importlib import reload
from tqdm import tqdm_notebook as tqdm
import time

import pdaactconn as pc
from trialexplorer import AACTStudySet

import matplotlib.pyplot
%matplotlib inline

## Create the StudySet object, if no conn as param, defaults to a local connection

In [ ]:
ss = AACTStudySet.AACTStudySet(tqdm_handler=tqdm)

### Alternatively can pass a explicit connection object to the class to connect to remote

In [ ]:
conn = pc.AACTConnection(source=pc.AACTConnection.REMOTE)
ss = AACTStudySet.AACTStudySet(conn=conn, 
                               tqdm_handler=tqdm)

### Can also set the connection directly

In [ ]:
ss.conn.set_source(conn.LOCAL)

### The study set starts with no data, we need to define our primary filter first

using the list_columns() function, we can see the initial columns that we can filter on

In [ ]:
ss.list_columns()[:10]

In [ ]:
len(ss.list_columns())

### All filters are added on a "AND" basis, but we can and "OR" filters by adding it as 1 filter

filters are added in SQL syntax

In [ ]:
ss.add_constraint("start_date >= '2018-01-01'")
ss.add_constraint("start_date <= '2018-12-31'")

In [ ]:
ss.show_constraints()

In [ ]:
# remove the 1st constraint (0 indexed)
ss.remove_constraint(0)
ss.show_constraints()

In [ ]:
# add it back
ss.add_constraint("start_date >= '2018-01-01'")
ss.show_constraints()

### Loads the studies from the main table into memory, saved in the self.studies dataframe

In [ ]:
ss.load_studies()

In [ ]:
ss.studies.head()

In [ ]:
ss.studies.shape

### To add dimensions, we need to know the names of the dimensions

This is loaded into this attribute on init, which includes a list of implemented dimensions

In [ ]:
ss.avail_dims.list

### The .avail_dims attribute also supports tab-completion of implemented attributes:

In [ ]:
ss.avail_dims.brief_summaries

### to add a dimension, we just need to add the key of the dimension using this method:

This will call the constructor of the dimension handler and link that Object to the StudySet object

In [ ]:
ss.add_dimensions(['brief_summaries', 'links', 'designs', 'design_outcomes'])

In [ ]:
ss.dimensions

### At init, it is empty, but we can call refresh_dim_data to load it from the database:

this uses the self.studies dataframe as the master list, and creates a temp table in the database with its nct_ids

it the joins the dimension tables to that temp table so we only load the needed data

In [ ]:
ss.refresh_dim_data()

### The dimensional data is loaded in the dim object

in 2 forms, raw_data, is the direct return from the database

In [ ]:
cur_dim = ss.dimensions['brief_summaries']

In [ ]:
cur_dim.raw_data.head()

this is split into a dict keyed by nct_id in .data for ease of use, these refer to the same mem location so do not use any more memory

In [ ]:
test_nct_id = ss.studies.index[5]
test_nct_id

In [ ]:
cur_dim.data[test_nct_id]

### can access other dims too

In [ ]:
cur_dim = ss.dimensions['design_outcomes']

In [ ]:
cur_dim.raw_data.head()

In [ ]:
cur_dim.data[test_nct_id]

### A key design choice is dropping records from the main studies table:

It will drop the records from the ss.studies table, and clean up any dimensions associated with it.

As a result it is a bit slow:

In [ ]:
to_drop = [ss.studies.index[5], ss.studies.index[6], ss.studies.index[7]]
to_drop

In [ ]:
ss.drop_studies(to_drop)

The memory reduction is key as the user iterates and modifies the active studies list, 

**so we should test the memory clearing functionality carefully!!**

### Testing dropping of dimensions

In [ ]:
to_drop = list(ss.dimensions.keys())
ss.drop_dimensions(to_drop)

In [ ]:
ss.dimensions

### Reloading the test case

In [2]:
reload(AACTStudySet)

tested_dims = ['result_groups', 'milestones']

ss = AACTStudySet.AACTStudySet(tqdm_handler=tqdm)
ss.add_constraint("start_date >= '2018-06-30'")
ss.add_constraint("start_date <= '2018-12-31'")
ss.load_studies()

ss.add_dimensions(tested_dims)


12790 studies loaded!
Successfuly added these 2 dimensions: ['interventions', 'intervention_other_names']
Failed to add these 0 dimensions: []


In [3]:
print(ss.dimensions['interventions']._generate_load_query())


                SELECT 
                    a.* 
                FROM interventions a
                JOIN temp_cur_studies tmp ON tmp.nct_id = a.nct_id 
                


In [5]:
print(ss.dimensions['intervention_other_names']._generate_load_query())


                SELECT 
                    a.* 
                FROM interventions b
                JOIN temp_cur_studies tmp ON tmp.nct_id = b.nct_id
                JOIN intervention_other_names a on a.nct_id=tmp.nct_id and b.id = a.intervention_id
                


In [ ]:
ss.refresh_dim_data()

In [ ]:
for k, cur_dim in ss.dimensions.items():
    print(k)
    display(ss.dimensions[cur_dim].raw_data.head(2))